In [1]:
#load the csv to the notebook
import pandas as pd
import datetime 

In [2]:
rawInput=pd.read_csv('lasvegasStrip.csv',encoding="ISO-8859-1")
rawInput.head()

,Unnamed: 0,user,location,date,rating,title,comments
0,0,s6steve,"Sheffield, United Kingdom","July 9, 2018",20,Over priced over rated,Over rated new Las Vegas has taken away the sp...
1,0,sathyanupheap,unknown,"July 9, 2018",40,Plenty to See...,I rented a Mini Cooper convertible and it was ...
2,0,Blu H,unknown,"July 9, 2018",50,City of anything goes!,If you haven¡¯t been here yet you¡¯re probably...
3,0,anita a,"Los Angeles, California","July 9, 2018",30,Not like it use to be,Back in the day you could eat and have fun on ...
4,0,kristycullinan,"Philadelphia, Pennsylvania","July 9, 2018",30,Walking walking and more walking.,The strip is 5 miles long and pretty spaced ou...


In [66]:
rawInput['realDate']=[datetime.datetime.strptime(i,'%B %d, %Y') for i in rawInput['date']]

In [69]:
rawInput=rawInput[rawInput['realDate']>datetime.datetime.strptime('January 01, 2014','%B %d, %Y')]

## preprocess text

In [2]:
#import the nltk lib for preprocessing
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

In [5]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [71]:
#nltk.download() #make sure the computer has all nltk corpus

In [72]:
import string #for getting punctuation
import gensim

In [73]:
stop = set(stopwords.words('english'))
excludeMyself=['put','thats','that','what','whats','vega','la','las','thing','soooo','so','soo','sooo','world','si','lot','ops','didnt','cant','some',
              'somehow','describe','name','whilst','even','actual','one','two','three','four','five','six','seven','eight','nine','ten','duck','amongst',
              'something','most','fully','what','whats','whatever','many','stip','strip','occasional','occasionally','wont','will','non','i','ive']
exclude = list(set(string.punctuation))
lemma = WordNetLemmatizer()##WordNetLemmatizer is a class

In [74]:
print (exclude)

[';', '-', ':', '>', '.', '$', '\\', ']', '(', '!', '@', '`', '}', '=', '<', '/', '+', '*', '_', ')', '{', '|', '#', "'", '&', '~', '%', '"', '[', '^', '?', ',']


In [75]:
##a def to clean the text(typical way), very nice
def clean(doc):
    stop_free = " ".join([i for i in gensim.utils.simple_preprocess(doc,min_len=3) if i not in stop])
    ##simple_preprocess can lowercase words and tokenize them,you can set up min len of token too,here is 3, so word like AI will not be included
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())##这个用来找stem
    myselfOutput = " ".join(word for word in normalized.split() if word not in excludeMyself)##这个用来找stem
    return myselfOutput

In [76]:
clean(rawInput['comments'][0])

'rated new taken away spark original prepared pant drink bother night club guy pretty sexist priced staff everywhere arrogant care long'

## here is the text dataset

In [77]:
doc_complete=rawInput['comments'][:]

In [78]:
doc_clean = [clean(doc).split() for doc in doc_complete]    
#doc_clean is a list中包list,每个item是一个doc的tokens

In [79]:
doc_clean[:2]

[['rated',
  'new',
  'taken',
  'away',
  'spark',
  'original',
  'prepared',
  'pant',
  'drink',
  'bother',
  'night',
  'club',
  'guy',
  'pretty',
  'sexist',
  'priced',
  'staff',
  'everywhere',
  'arrogant',
  'care',
  'long'],
 ['rented',
  'mini',
  'cooper',
  'convertible',
  'nice',
  'cruise',
  'around',
  'light',
  'sight',
  'sound',
  'crowd',
  'city',
  'never',
  'sleep',
  'everyone',
  'young',
  'old']]

## create gensim corpus dicts and doc_term_Matrix

In [80]:
# Importing Gensim
import gensim
from gensim import corpora

In [81]:
# Creating the term dictionary of our courpus, where every unique term（token) is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

In [82]:
print(len(dictionary))
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

15988
0 arrogant
1 away
2 bother
3 care
4 club
5 drink
6 everywhere
7 guy
8 long
9 new
10 night


In [83]:
#### filter_extremes可以把出现less than 15 documents的tokens (absolute number) 
#or more than 0.5 documents (fraction of total dictionary size, not absolute number). dictionary里有一半都是那个词删掉。
#after the above two steps, keep only the first 100000 most frequent tokens.
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000) #当文字很多的时候用这个,先不run

In [84]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [85]:
print (doc_clean[0])

['rated', 'new', 'taken', 'away', 'spark', 'original', 'prepared', 'pant', 'drink', 'bother', 'night', 'club', 'guy', 'pretty', 'sexist', 'priced', 'staff', 'everywhere', 'arrogant', 'care', 'long']


In [86]:
print (doc_term_matrix[0])###这个matrix是每个doc的list of (token_id, token_count) tuples

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]


In [87]:
print (dictionary[0])#dictionary 可以直接输入token的id得出token
dictionary[doc_term_matrix[0][2][0]]#找到第一个id为0的词

arrogant


'bother'

## tf-idf model LDA
create tf-idf scores, 这个score就是给每个token一个score，这score的好处是：当很多doc都有某个词比如city，那么city对选择topic意义并不大，当有些词是某个doc独有如accessible，那么这个词的意义还更强。。
https://rare-technologies.com/pivoted-document-length-normalisation/

In [88]:
from gensim import corpora, models
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

In [89]:
corpus_tfidf[0]##第一个doc里面词的score，可以看到这个score跟doc_term_matrix是不一样的,出现太多的词score小

[(0, 0.396509863470022),
 (1, 0.1382966948741414),
 (2, 0.20752021014663408),
 (3, 0.20716030464473467),
 (4, 0.14524124598088017),
 (5, 0.10239923561973541),
 (6, 0.1248495840705397),
 (7, 0.17155354217631966),
 (8, 0.10714552744349849),
 (9, 0.10362914064347638),
 (10, 0.055886188322609706),
 (11, 0.24286956830418602),
 (12, 0.30028425774279366),
 (13, 0.1382966948741414),
 (14, 0.13849040013105807),
 (15, 0.1973549061646629),
 (16, 0.23497574265160948),
 (17, 0.3686944254631472),
 (18, 0.3686944254631472),
 (19, 0.22489069708533646),
 (20, 0.17793668024085066)]

In [90]:
lda_model_tfidf = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=15, id2word=dictionary, passes=50)

In [91]:
lda_model_tfidf.show_topics(num_topics=15, num_words=20)##可以发现tfidf去掉了很多general的词。

[(0,
  '0.007*"length" + 0.006*"colour" + 0.005*"pair" + 0.005*"fashion" + 0.005*"centre" + 0.005*"pain" + 0.005*"craziness" + 0.005*"accessible" + 0.005*"pass" + 0.004*"transport" + 0.004*"odd" + 0.004*"anymore" + 0.004*"travelled" + 0.004*"drunken" + 0.004*"hurry" + 0.004*"sdx" + 0.004*"golden" + 0.004*"glitter" + 0.004*"cab" + 0.004*"boutique"'),
 (1,
  '0.008*"change" + 0.007*"circus" + 0.005*"disappoint" + 0.005*"completely" + 0.005*"property" + 0.005*"eatery" + 0.005*"allow" + 0.004*"dance" + 0.004*"bother" + 0.004*"round" + 0.004*"linq" + 0.004*"lined" + 0.004*"honestly" + 0.004*"whenever" + 0.004*"seriously" + 0.004*"sleeping" + 0.004*"morning" + 0.004*"advantage" + 0.004*"newer" + 0.003*"house"'),
 (2,
  '0.006*"carlo" + 0.006*"monte" + 0.006*"pushy" + 0.005*"hawker" + 0.005*"behind" + 0.004*"lighting" + 0.004*"presence" + 0.004*"firework" + 0.004*"watched" + 0.004*"unsafe" + 0.004*"airport" + 0.004*"east" + 0.004*"daylight" + 0.004*"eve" + 0.003*"circus" + 0.003*"timeshare" +

## extract doc topics

In [75]:
print (doc_clean[0])
print (doc_term_matrix[0])

['rated', 'new', 'la', 'vega', 'taken', 'away', 'spark', 'original', 'strip', 'prepared', 'pant', 'drink', 'bother', 'night', 'club', 'guy', 'pretty', 'sexist', 'priced', 'staff', 'everywhere', 'arrogant', 'care', 'long']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)]


In [76]:
lda_model_tfidf.get_document_topics(doc_term_matrix[0])

[(4, 0.1440727),
 (10, 0.041333333),
 (16, 0.045069337),
 (19, 0.08133333),
 (20, 0.47684383),
 (27, 0.1793475)]

## typical LDA model(bag of words)

In [63]:
# Creating the object for LDA model using gensim library
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=5, id2word=dictionary, passes=2, workers=2)
#这里workers是用几个cpu的意思，multicore就是多核运算意思

In [64]:
print(lda_model.print_topics(num_topics=5, num_words=10))

[(0, '0.046*"vega" + 0.024*"casino" + 0.023*"la" + 0.022*"people" + 0.018*"hotel" + 0.018*"lot" + 0.017*"fun" + 0.016*"one" + 0.016*"place" + 0.016*"great"'), (1, '0.036*"people" + 0.031*"night" + 0.029*"see" + 0.022*"walk" + 0.021*"light" + 0.020*"walking" + 0.018*"get" + 0.017*"lot" + 0.016*"casino" + 0.014*"day"'), (2, '0.034*"vega" + 0.024*"hotel" + 0.016*"walking" + 0.016*"like" + 0.014*"place" + 0.013*"day" + 0.013*"la" + 0.012*"lot" + 0.012*"see" + 0.012*"one"'), (3, '0.036*"see" + 0.025*"place" + 0.024*"hotel" + 0.022*"walk" + 0.019*"time" + 0.019*"vega" + 0.019*"much" + 0.016*"casino" + 0.013*"show" + 0.013*"walking"'), (4, '0.044*"vega" + 0.032*"time" + 0.030*"people" + 0.020*"day" + 0.018*"hotel" + 0.017*"show" + 0.015*"la" + 0.015*"take" + 0.014*"like" + 0.014*"walking"')]


In [65]:
lda_model.show_topics(num_topics=5, num_words=10)

[(0,
  '0.046*"vega" + 0.024*"casino" + 0.023*"la" + 0.022*"people" + 0.018*"hotel" + 0.018*"lot" + 0.017*"fun" + 0.016*"one" + 0.016*"place" + 0.016*"great"'),
 (1,
  '0.036*"people" + 0.031*"night" + 0.029*"see" + 0.022*"walk" + 0.021*"light" + 0.020*"walking" + 0.018*"get" + 0.017*"lot" + 0.016*"casino" + 0.014*"day"'),
 (2,
  '0.034*"vega" + 0.024*"hotel" + 0.016*"walking" + 0.016*"like" + 0.014*"place" + 0.013*"day" + 0.013*"la" + 0.012*"lot" + 0.012*"see" + 0.012*"one"'),
 (3,
  '0.036*"see" + 0.025*"place" + 0.024*"hotel" + 0.022*"walk" + 0.019*"time" + 0.019*"vega" + 0.019*"much" + 0.016*"casino" + 0.013*"show" + 0.013*"walking"'),
 (4,
  '0.044*"vega" + 0.032*"time" + 0.030*"people" + 0.020*"day" + 0.018*"hotel" + 0.017*"show" + 0.015*"la" + 0.015*"take" + 0.014*"like" + 0.014*"walking"')]

## pyLDAvis  
dictionary/lda/doc_term_matrix可以先保存,然后load的

In [77]:
"""
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
doc_term_matrix = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
"""

"\ndictionary = gensim.corpora.Dictionary.load('dictionary.gensim')\ndoc_term_matrix = pickle.load(open('corpus.pkl', 'rb'))\nlda = gensim.models.ldamodel.LdaModel.load('model5.gensim')\n"

In [92]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda_model_tfidf, doc_term_matrix, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)